## Example Active Learning Run (Pool-based)

Inialize pool and test data set

In [ ]:
from alef.oracles.flexible_oracle import Flexible1DOracle
import numpy as np

# here we simulate a dataset - either you can plug in an own dataset or use the dataset classes of alef
# configure function that should be learned actively
# f = lambda x: 0.1*np.sin(30*x)+2*x-1.0
f = lambda x: (x - 0.75) * 10.0 * np.exp(-1 * np.power((x - 0.75) * 10.0, 2.0))
f = lambda x: x + 0.2 * np.sin(40 * x)

# Oracle configuration
test_function = Flexible1DOracle()
test_function.set_box_bounds(0.0, 1.0)
test_function.set_f(f)
test_function.plot()
x_gt, f_gt = test_function.get_random_data(1000, False)
x_data_pool, y_data_pool = test_function.get_random_data(2000)
x_test, y_test = test_function.get_random_data(200)
n_dim = x_data_pool.shape[1]

Configurations of model and active learner

In [2]:
from alef.configs.models.gp_model_config import (
    BasicGPModelConfig,
)
from alef.configs.active_learner.active_learner_configs import (
    RandomActiveLearnerConfig,
    PredSigmaActiveLearnerConfig,
)
from alef.configs.kernels.hhk_configs import HHKTwoLocalDefaultConfig
from alef.enums.active_learner_enums import ValidationType

# Model configuration
kernel_config = HHKTwoLocalDefaultConfig(input_dimension=1)
model_config = BasicGPModelConfig(kernel_config=kernel_config)
# model_config = BasicGPModelMarginalizedConfig(kernel_config=kernel_config)

# Active Learning configuration
active_learner_config = PredSigmaActiveLearnerConfig()
active_learner_config.use_smaller_acquistion_set = False
n_initial_datapoints = 5
n_al_steps = 20
initial_data_seed = 10
do_plotting = True
FIT_GP_HYPERPARAMETER_AT_START = True
SET_INITIAL_LOCATIONS_MANUALLY = True
initial_xs = [0.2, 0.5, 0.8]

# Random Learning configuration (for comparision)
random_learning_config = RandomActiveLearnerConfig()
assert random_learning_config.validation_type == active_learner_config.validation_type
validation_type_name = ValidationType.get_name(active_learner_config.validation_type)

Initialize model and active learner

In [ ]:
from alef.models.gp_model import GPModel
from alef.models.model_factory import ModelFactory
from alef.active_learner.active_learner_factory import ActiveLearnerFactory

model = ModelFactory.build(model_config)
if isinstance(model, GPModel) and FIT_GP_HYPERPARAMETER_AT_START:
    x_data, y_data = test_function.get_random_data(500)
    model.infer(x_data, y_data)
    model.fix_current_hyperparameters()

active_learner = ActiveLearnerFactory.build(active_learner_config)
active_learner.set_model(model)
active_learner.set_ground_truth(x_gt, f_gt)
active_learner.set_pool(x_data_pool, y_data_pool)

if SET_INITIAL_LOCATIONS_MANUALLY:
    initial_ys = []
    for initial_x in initial_xs:
        initial_y = test_function.query(initial_x)
        initial_ys.append(initial_y)
    initial_xs = np.expand_dims(np.array(initial_xs), axis=1)
    initial_ys = np.expand_dims(np.array(initial_ys), axis=1)
    active_learner.set_initial_dataset_manually(initial_xs, initial_ys)
else:
    active_learner.sample_initial_data(n_initial_datapoints, initial_data_seed, set_seed=True)
active_learner.set_test_set(x_test, y_test)
active_learner.set_do_plotting(do_plotting)

Run active learner

In [ ]:
validation_metrics_al, _ = active_learner.learn(n_al_steps)

Initialize random learner

In [5]:
if not (isinstance(model, GPModel) and FIT_GP_HYPERPARAMETER_AT_START):
    model = ModelFactory.build(model_config)
random_learner = ActiveLearnerFactory.build(random_learning_config)
random_learner.set_model(model)
random_learner.set_pool(x_data_pool, y_data_pool)
random_learner.sample_initial_data(n_initial_datapoints, initial_data_seed, set_seed=True)
random_learner.set_test_set(x_test, y_test)
random_learner.set_do_plotting(do_plotting)

Run random learner

In [ ]:
validation_metrics_random, _ = random_learner.learn(n_al_steps)

Create learning curve plots

In [ ]:
from alef.utils.metric_curve_plotter import MetricCurvePlotter
import numpy as np

curve_plotter = MetricCurvePlotter(1)
x_axis = np.arange(0, n_al_steps)
curve_plotter.add_metrics_curve(x_axis, validation_metrics_random["RMSE"], "blue", "random", 0)
curve_plotter.add_metrics_curve(x_axis, validation_metrics_al["RMSE"], "red", "active", 0)
curve_plotter.configure_axes(0, validation_type_name, "Iteration", validation_type_name, True, True)
curve_plotter.show()